In [ ]:
!pip install ics icalendar

In [245]:
from ics import Calendar, Event
# from ics.timezone import UTC, is_utc
c = Calendar()
e = Event()
e.name = "My cool event"
e.begin = '2014-01-01 00:00:00'
c.events.add(e)
c.events # [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
# with open('data/ignored/my.ics', 'w') as my_file:
#     my_file.writelines(c.serialize_iter())

{<Event 'My cool event' begin:2014-01-01T00:00:00+00:00 end:2014-01-01T00:00:00+00:00>}

In [250]:
import icalendar
cal = icalendar.Calendar()
cal[""]
event = icalendar.Event()
event.add('summary', "My cool event")
event.add('dtstart', datetime.strptime("01-01-2024 12:00", strp).astimezone(timezone.utc))
event.add('dtend', datetime.strptime("01-01-2024 13:00", strp).astimezone(timezone.utc))
# event.add('dtstamp', module.closing_time)
# event['uid'] = "module/" + str(module.id) + "/A+"
cal.add_component(event)
cal.to_ical()


b'BEGIN:VCALENDAR\r\nBEGIN:VEVENT\r\nSUMMARY:My cool event\r\nDTSTART:20240101T110000Z\r\nDTEND:20240101T120000Z\r\nEND:VEVENT\r\nEND:VCALENDAR\r\n'

## Create calendar from PDF table

In [155]:
from pathlib import Path
base_path = Path("/Users/az/Documents/repos/Scripts/python_packages/data/ignored/raport3/")
i = 3
src_path = base_path / f"plan {i}.pdf"
raw_path = base_path / f"raport{i}_raw.txt"
man_path = base_path / f"raport{i}_manual_review.txt"
tgt_path = base_path / f"raport.ics"

In [142]:
# From PDF
from unstructured.partition.auto import partition

elements = partition(filename=str(src_path))

with open(raw_path, "w") as fh:
    for e in elements:
        fh.write(str(e) + "\n")

In [86]:
IGONRED_LINES = (
    "RPS", "data/blok", "rodz.", "kierunek: Lekarski", "przedmiot", "PLAN ZAJĘĆ", "tryb. stac.", "rodzaj", "grupa", "wybór", "sala", "prowadzący", "Data Wydruku", "Koniec grupy", "SJM",
)

with open(raw_path, "r") as fh:
    content = fh.readlines()
    content = [x.strip() for x in content if x.strip() and not x.strip().startswith(IGONRED_LINES)]
    rok_id = content.index("rok")
    del content[rok_id:rok_id+4 + 1]
    content = [x.replace("ONLINE", "|ONLINE")
               .replace("WSS", "|WSS")
               .replace("FB_3", "|FB_3")
               .replace("USK", "|USK")
               .replace("Rad_2_nr_4", "|Rad_2_nr_4")
               .replace("CSM_116_P", "|CSM_116_P")
               .replace("Rad_9_nr_2", "|Rad_9_nr_2")
               .replace("FB_5", "|FB_5")
               for x in content]

new_content = []
for c in content:
    new_content.extend(c.split("|"))
new_content = [x.strip() for x in new_content if x.strip()]

with open(man_path, "w") as fh:
    for line in new_content:
        fh.write(line + "\n")

In [143]:
with open(man_path, "r") as fh:
    content = fh.readlines()
    content = [x.strip() for x in content if x.strip()]

In [87]:
from dataclasses import dataclass

@dataclass
class Day:
    day: str

    @staticmethod
    def is_day(text):
        return text.endswith(("Pon.", "Wt.", "Śr.", "Czw.", "Pt.", "Sb.", "Nd."))
    
@dataclass
class Type:
    type: str

    @staticmethod
    def is_type(text):
        return text in ("Sem", "Ćwi", "Wyk")

@dataclass
class Hour:
    hour: str

@dataclass
class Details:
    day: Day
    type: Type
    hour: Hour

In [144]:
for i, c in enumerate(content):
    if isinstance(c, str) and Day.is_day(c):
        content[i] = Day(c)

In [145]:
for i, c in enumerate(content):
    if isinstance(c, str) and Type.is_type(c):
        content[i] = Type(c)
        content[i-1] = Hour(content[i-1])

In [146]:
# Replace 
# [
#     Day1
#     Hour1
#     Type1
#     Hour2
#     Type2
# ]
# with
# [
#     Details(Day1, Type1, Hour1)
#     Details(Day1, Type2, Hour2)
# ]
new_content = []
last_day = None
for i, c in enumerate(content):
    if isinstance(c, Day):
        last_day = c
    elif isinstance(c, Hour):
        new_content.append(Details(day=last_day, type=content[i+1], hour=c))
    elif isinstance(c, Type):
        pass
    else:
        new_content.append(c)
content = new_content

In [147]:
from itertools import groupby

new_content = []
for key, group in groupby(content, key=type):        # aggragate content list into groups of (n x Details + 4n x str) x 12 pages
    if key == Details:
        new_content.append(list(group))              # day, hour, type
    else:
        length = len(new_content[-1])
        items = list(group)
        assert len(items) == 4 * length, f"Expected 4 x {length}, got {len(items)}, last day: {new_content[-1][-1]}"
        new_content.append(items[:length])           # name
        new_content.append(items[length:2*length])   # group
        new_content.append(items[2*length:3*length]) # place
        new_content.append(items[3*length:])         # teacher
content = new_content
# content

In [148]:
# Each batch contains [[details1, details2, ...], [name1, name2, ...], [group1, group2, ...], [place1, place2, ...], [teacher1, teacher2, ...]]
n = 5
batches = [content[start:end] for start, end in zip(range(0, len(content), n), range(n, len(content) + n, n))]
# batches

In [149]:
from __future__ import annotations

import pytz
from datetime import datetime, timezone
from ics import Calendar, Event

strp = "%d-%m-%Y %H:%M"

@dataclass
class Class:
    day: str
    type: str
    hour: str
    name: str
    group: str
    place: str
    teacher: str

    def __post_init__(self):
        self.day = self.day.split(' ')[0]
        try:
            start_hour, end_hour = self.hour.split("-")
            tz = pytz.timezone('Europe/Warsaw')
            self.start_hour = datetime.strptime(f"{self.day} {start_hour}", strp).astimezone(timezone.utc)
            self.end_hour = datetime.strptime(f"{self.day} {end_hour}", strp).astimezone(timezone.utc)
        except Exception as e:
            print("ERROR ", self.hour, self.day, e)

    def matches_on_time_and_group(self, other: Class) -> bool:
        return self.day == other.day and self.hour == other.hour and self.group == other.group

    def to_event(self):
        e = Event()
        e.name = f"{self.name} ({self.group})"
        e.description = f"Prowadzący: {self.teacher}\nMiejsce: {self.place}"
        e.begin = self.start_hour
        e.end = self.end_hour
        return e

In [150]:
classes = []
for batch in batches:
    for a, b, c, d, e in zip(*batch):
        classes.append(Class(day=a.day.day, type=a.type.type, hour=a.hour.hour, name=b, group=c, place=d, teacher=e))
# classes

In [151]:
# Remove duplicated entries on page division
new_classes = [classes[0]]
for c in classes[1:]:
    prev_c = new_classes[-1]
    if c.matches_on_time_and_group(prev_c):
        # print(f"Joined classes:\n{prev_c}\n{c}\n")
        if c.place != prev_c.place:
            new_classes[-1].place = f"{prev_c.place} {c.place}"
        if c.name != prev_c.name:
            new_classes[-1].name = f"{prev_c.name} {c.name}"
    else:
        new_classes.append(c)
classes = new_classes

In [152]:
calendar = Calendar()

In [153]:
for c in classes:
    calendar.events.add(c.to_event())

In [154]:
with open(tgt_path, 'w') as fh:
    fh.writelines(calendar.serialize_iter())